In [4]:
import numba as nb
import numpy as np

In [13]:
@nb.njit(nb.types.UniTuple(nb.int64[:], 2)(nb.types.ListType(nb.types.ListType(nb.int64))))
def _jit_serialize(item):

    n_values=0
    n_starts=0
    for segment in item:
        n_values+=len(segment)
        n_starts+=1

    values = np.empty((n_values), dtype=nb.int64)
    starts = np.empty((n_starts+1), dtype=nb.int64)

    counter=0
    for ii,segment in enumerate(item):
        starts[ii]=counter
        for jj in sorted(segment):
            values[counter]=jj
            counter+=1

    starts[-1]=counter

    return starts, values

In [14]:
item = {2:[3,4,5],7:[1,10],8:[3,4,6,7],3:[8],6:[2,9,1]}
indices = np.sort(list(item.keys()))
aux_item = nb.typed.List([nb.typed.List(sorted(item[ii])) for ii in indices])

In [15]:
_jit_serialize(aux_item)

(array([ 0,  3,  4,  7,  9, 13]),
 array([ 3,  4,  5,  8,  1,  2,  9,  1, 10,  3,  4,  6,  7]))